In [1]:
from utils import compute_f1, load_data
from text_train.imports import *
from text_train.data import TextDS, text_to_device
from arcface import ArcMarginProduct
from text_train.model import EMBBert
from text_train.train import *
np.random.seed(1337)
device = torch.device('cuda')

In [2]:
df, train_df, val_df, train_labels, val_labels = load_data(train_perc=1.0,df_path = '/home/dlo/Documents/product_match/data/train.csv')

cleaned data


In [3]:
bs = 64
bs_val = 128
max_length=200
agg = 'mean'
language_model = 'bert-base-multilingual-uncased'
tokenizer = BertTokenizer.from_pretrained(language_model)

In [4]:

tr_ds = TextDS(train_df, tokenizer, max_length=max_length)
#val_ds = TextDS(val_df, tokenizer, max_length=max_length)
#full_ds = TextDS(df, tokenizer, max_length=max_length)

In [7]:
tr_test_dl = DataLoader(tr_ds, batch_size = bs_val, shuffle = False, pin_memory = True)
tr_dl = DataLoader(tr_ds, batch_size = bs, shuffle = True, pin_memory = True)
#val_dl = DataLoader(val_ds, batch_size = bs_val, shuffle = False, pin_memory = True)

In [6]:
model = EMBBert(language_model, agg=agg).to(device)

In [8]:
centers = compute_centers(tr_test_dl, model, train_df)

  0%|          | 0/268 [00:00<?, ?it/s]

In [8]:
torch.save(centers, 'centers_bert.pt')

In [7]:
centers = torch.load('centers_bert.pt')

In [9]:

metric_fc = ArcMarginProduct(512, train_df['label_group'].nunique(), s=30, m=0.5,
                             easy_margin=False, centers=centers, device=device).to(device)

Using center as wieghts


In [10]:
mnp = list(model.named_parameters())

param_groups = [{'params' : [p for n,p in mnp if 'embeddings' in n]}]

n_blocks = 11
for i in range(n_blocks):
    ith_block = [p for n, p in mnp if f'layer.{i}.' in n]
    param_groups.append({'params' : ith_block})
    
param_groups.append({'params' : [p for n,p in mnp if n in ['text_embedor.pooler.dense.weight', 
                                                     'text_embedor.pooler.dense.bias']]})

param_groups.append({'params' : metric_fc.parameters()})

In [11]:
lf = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(param_groups, weight_decay = 1e-5)

lr = list(np.linspace(1e-5,3e-4,len(param_groups)))

n_epochs = 10
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr, pct_start=0.3,
                                            total_steps=int(n_epochs * len(tr_dl)))

In [12]:

loss_hist = []
best_thr_score={'val': {'thr': None, 'f1': None}, 'train': {'thr': None, 'f1': None}}
thr_score_hist=[]
ep_start = 0
save_path = 'data/text_models/test_12ap_epochs'

In [ ]:
best_thr_score, thr_score_hist, losses = train_full_data(model, optimizer, lf, sched, metric_fc, tr_dl,
                                               n_epochs, train_df, 
                                               save_path=save_path, 
                                               prev_best_info=best_thr_score, info_history=thr_score_hist,
                                               ep_start=ep_start)
loss_hist.append(losses)
ep_start += n_epochs

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/536 [00:00<?, ?it/s]